In [1]:
# This code runs on Julia 1.94, Oscar v0.12.1, and Polymake v0.9.2
using Oscar 
using Polymake

 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.12.1 ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2023 by The OSCAR Development Team


In [13]:
# Generate polytope
c = polytope.cyclic(2, 6) 

# Get vertex-facet incidence matrix of c
m = c.VERTICES_IN_FACETS

# n is the number of facets c
n = size(m)[1]

6

In [14]:
# Generate a list of lists where the ith list contains faces which do not intersect the ith face
non_intersecting_facets = []
for i in 1:n
    # this list will contain facets that do not intersect the ith face
    i_non_neighbors = []
    # Check if the jth face and ith face have an common vertices
    for j in 1:n
        empty = true                    # intersection is empty
        # Check all vertices...
        for k in 1:size(m)[2]  
            #... to see if they lie in both the i face and the j face
            if m[i, k] && m[j, k]
                # If the k vertex lies in both the i and j faces, then the intersection is non-empty
                empty = false
                break
            end
        end
        # If i and j have no vertices in common, their intersection is non-empty, then add j to the list of non neighbors of i
        if empty
            push!(i_non_neighbors, j)
        end
    end
    # Add the list of facets that do not intersect i to the list of non-intersecting faces
    push!(non_intersecting_facets, i_non_neighbors)
end

In [36]:
# Create a ring from the variables
variables = [join(["u", string(i)]) for i in 1:n]
R, u = PolynomialRing(QQ, variables)

# Our list of u-equations will be stored in ueqs
ueqs = []

# Set all the exponents to be 1
exponents = [1 for i in non_intersecting_facets]

# Set the exponents manually
# exponents = [2, 3, 1, 1, 1, 1]

# For each i, add the corresponding u equation to the list
for i in 1:n
    product = prod([u[j]^exponents[i] for j in non_intersecting_facets[i]])
    push!(ueqs, u[i] + product - 1)
end

# Create an ideal from the u equations
I = Oscar.ideal(ueqs)

# Print the dimension: this much match the dimension of the original polytope to have home of a binary geometry
display(ueqs)
print(dim(I), "\n")


6-element Vector{Any}:
 u1 + u3*u4*u5 - 1
 u2 + u4*u5*u6 - 1
 u1*u5*u6 + u3 - 1
 u1*u2*u6 + u4 - 1
 u1*u2*u3 + u5 - 1
 u2*u3*u4 + u6 - 1

1
